In [1]:
## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Wilmington,US,34.2257,-77.9447,42.01,81,1,3.44,clear sky
1,1,East London,ZA,-33.0153,27.9116,91.40,49,0,4.61,clear sky
2,2,Port Alfred,ZA,-33.5906,26.8910,84.88,60,44,6.64,scattered clouds
3,3,Saskylakh,RU,71.9167,114.0833,-23.10,85,100,3.36,overcast clouds
4,4,Dedovichi,RU,57.5517,29.9502,35.15,98,100,9.04,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 75


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Hilo,US,19.7297,-155.0900,69.80,83,40,4.61,light rain
13,13,Ixtapa,MX,20.7000,-105.2000,68.00,94,34,2.77,scattered clouds
19,19,Umm Kaddadah,SD,13.6017,26.6876,72.05,21,11,16.46,few clouds
24,24,Kuito,AO,-12.3833,16.9333,71.17,51,69,6.06,broken clouds
29,29,Sao Filipe,CV,14.8961,-24.4956,74.97,60,99,9.71,overcast clouds
31,31,Kapaa,US,22.0752,-159.3190,73.40,83,75,19.57,light rain
32,32,Mar Del Plata,AR,-38.0023,-57.5575,70.00,100,75,6.91,mist
33,33,Henties Bay,NaN,-22.1160,14.2845,69.01,93,100,7.36,light rain
36,36,Kasongo-Lunda,CD,-6.4667,16.8167,72.45,90,100,2.08,light rain
41,41,Kaeo,NZ,-35.1000,173.7833,68.00,63,3,5.01,clear sky


In [6]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                121
City                   121
Country                118
Lat                    121
Lng                    121
Max Temp               121
Humidity               121
Cloudiness             121
Wind Speed             121
Current Description    121
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
nan_value = float("NaN")
clean_df.replace("", nan_value, inplace = True)
clean_df.dropna(subset = ["Country"], inplace = True)
clean_df

C:\Users\Dr. Awesome\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4385: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
C:\Users\Dr. Awesome\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
11,11,Hilo,US,19.7297,-155.0900,69.80,83,40,4.61,light rain
13,13,Ixtapa,MX,20.7000,-105.2000,68.00,94,34,2.77,scattered clouds
19,19,Umm Kaddadah,SD,13.6017,26.6876,72.05,21,11,16.46,few clouds
24,24,Kuito,AO,-12.3833,16.9333,71.17,51,69,6.06,broken clouds
29,29,Sao Filipe,CV,14.8961,-24.4956,74.97,60,99,9.71,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
659,659,Caluquembe,AO,-13.7833,14.6833,71.17,29,100,6.58,overcast clouds
660,660,Sao Joao Da Barra,BR,-21.6403,-41.0511,73.56,85,100,5.57,overcast clouds
663,663,Bandar-E Lengeh,IR,26.5579,54.8807,66.20,37,1,21.85,sand
674,674,Valente,BR,-11.4122,-39.4619,66.24,98,89,4.97,overcast clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Hilo,US,69.80,light rain,19.7297,-155.0900,
13,Ixtapa,MX,68.00,scattered clouds,20.7000,-105.2000,
19,Umm Kaddadah,SD,72.05,few clouds,13.6017,26.6876,
24,Kuito,AO,71.17,broken clouds,-12.3833,16.9333,
29,Sao Filipe,CV,74.97,overcast clouds,14.8961,-24.4956,
...,...,...,...,...,...,...,...
659,Caluquembe,AO,71.17,overcast clouds,-13.7833,14.6833,
660,Sao Joao Da Barra,BR,73.56,overcast clouds,-21.6403,-41.0511,
663,Bandar-E Lengeh,IR,66.20,sand,26.5579,54.8807,
674,Valente,BR,66.24,overcast clouds,-11.4122,-39.4619,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("hotel not found... skipping")
hotel_df

hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping
hotel not found... skipping


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Hilo,US,69.80,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Ixtapa,MX,68.00,scattered clouds,20.7000,-105.2000,Container Inn Puerto Vallarta
19,Umm Kaddadah,SD,72.05,few clouds,13.6017,26.6876,
24,Kuito,AO,71.17,broken clouds,-12.3833,16.9333,Herdeiros de Antunes Capusso
29,Sao Filipe,CV,74.97,overcast clouds,14.8961,-24.4956,Tortuga B&B
...,...,...,...,...,...,...,...
659,Caluquembe,AO,71.17,overcast clouds,-13.7833,14.6833,
660,Sao Joao Da Barra,BR,73.56,overcast clouds,-21.6403,-41.0511,Pousada Porto De Canoas
663,Bandar-E Lengeh,IR,66.20,sand,26.5579,54.8807,کافی نت پارس
674,Valente,BR,66.24,overcast clouds,-11.4122,-39.4619,Bel Pousada E Restaurante


In [10]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace = True)
hotel_df.dropna(subset = ["Hotel Name"], inplace = True)

hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
11,Hilo,US,69.80,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
13,Ixtapa,MX,68.00,scattered clouds,20.7000,-105.2000,Container Inn Puerto Vallarta
24,Kuito,AO,71.17,broken clouds,-12.3833,16.9333,Herdeiros de Antunes Capusso
29,Sao Filipe,CV,74.97,overcast clouds,14.8961,-24.4956,Tortuga B&B
31,Kapaa,US,73.40,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
...,...,...,...,...,...,...,...
656,Mbeya,TZ,69.80,light rain,-8.9000,33.4500,Leda Lodge
660,Sao Joao Da Barra,BR,73.56,overcast clouds,-21.6403,-41.0511,Pousada Porto De Canoas
663,Bandar-E Lengeh,IR,66.20,sand,26.5579,54.8807,کافی نت پارس
674,Valente,BR,66.24,overcast clouds,-11.4122,-39.4619,Bel Pousada E Restaurante


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))